# Import and export data.

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Import dataset.

In [ ]:
# Files
dname = '/media/jacques/DATA/2019_MusicPred/experimentContinuousMatrix/data/behavior/'
fnames = glob.glob(dname + '*.xpd')

# Import dataframe
var_names = ('suj', 'condition', 'i_seq', 'bloc', 'trial', 'i_tone', 'decision', 'resp', 'rt', 'accurate')
DF = pd.DataFrame(columns=var_names)
for i, fname in enumerate(fnames):
    data = np.loadtxt(fname, skiprows=12, delimiter=',', 
                      dtype={'formats': ('i4', 'S12', 'i4', 'i4', 'i4', 'i4', 'S10', 'S10', 'S10', 'S10'), 
                             'names': var_names})
    _df = pd.DataFrame(data, columns=var_names)
    #_df.loc[:, 'suj'] += i
    DF = DF.append(_df, ignore_index=True)

In [ ]:
fnames

In [ ]:
DF = DF.loc[DF['condition'] == 'experimental']

In [ ]:
DF.loc[DF['decision'] == 'True', 'decision'] = True
DF.loc[DF['decision'] == 'None', 'decision'] = False
DF.loc[DF['resp'] == 'None', 'resp'] = np.nan
DF.loc[DF['rt'] == 'None', 'rt'] = np.nan

In [ ]:
DF['difficulty'] = np.mod(DF['i_seq'], 2) + 1

### Get dimensions.

In [ ]:
sujs = DF['suj'].unique()
blocs = DF['bloc'].unique()
i_seqs = DF['i_seq'].unique()
n_sujs = sujs.size
n_blocs = blocs.size
n_seqs = i_seqs.size
N_VOC = DF['i_tone'].unique().size
n_tones = DF['trial'].unique().size
n_dec = np.sum(DF.loc[DF['suj'] == sujs[-1], 'decision']) / n_seqs
print(n_sujs, n_blocs, n_seqs, n_tones, N_VOC, n_dec)

### Build arrays of data.

In [ ]:
MUSICIANS = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                      0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0], dtype=np.bool)
MUSICIANS = np.tile(MUSICIANS, (n_tones, n_seqs, 1)).T
MUSICIANS.shape

In [ ]:
CHOICES = np.full((n_sujs, n_seqs, n_dec), np.nan) # Choice made by the participant
DEC = np.full((n_sujs, n_seqs, n_tones), np.nan) # Whether the tones "ask" a decision or not
SEQS = np.full((n_sujs, n_seqs, n_tones), np.nan) # The sequence of played tones
PERF = np.full((n_sujs, n_seqs, n_dec), np.nan) # Whether the participant made a correct choice or not
RT = np.full((n_sujs, n_seqs, n_dec), np.nan) # Reaction time
SEQ_ID = np.full((n_sujs, n_seqs), np.nan) # Sequence identity
SEQS.shape

In [ ]:
for s, suj in enumerate(sujs):
    for i, i_seq in enumerate(DF.loc[DF['suj'] == suj, 'i_seq'].unique()):

        # Retrieve sequence
        select = np.logical_and(DF['i_seq'] == i_seq, DF['suj'] == suj)
        seq = np.array(DF.loc[select, 'i_tone'], dtype=np.int)
        if seq.size == 0: continue
        
        # Sequence
        SEQS[s, i] = seq.copy()
        SEQ_ID[s, i] = i_seq
        
        # Decision
        idx_pred = np.array(DF.loc[select, 'decision'], dtype=np.bool)
        DEC[s, i] = idx_pred.copy()
        
        # Choices
        choices = np.array(DF.loc[select, 'resp'], dtype=np.float)
        CHOICES[s, i] = choices[idx_pred].copy()

        # Response time
        RT[s, i] = np.array(DF.loc[select, 'rt'], dtype=np.float)[idx_pred]

        # Performance
        if seq[idx_pred].size == 0: continue
        PERF[s, i] = seq[idx_pred] == choices[idx_pred]
        PERF[s, i][np.isnan(choices[idx_pred])] = np.nan

### Save all arrays.

In [ ]:
np.save(dname + 'MUSICIANS', MUSICIANS)
np.save(dname + 'CHOICES', CHOICES)
np.save(dname + 'DEC', DEC)
np.save(dname + 'SEQS', SEQS)
np.save(dname + 'PERF', PERF)
np.save(dname + 'RT', RT)
np.save(dname + 'SEQ_ID', SEQ_ID)
np.save(dname + 'sujs', sujs)